# Dreambooth Stable Diffusion 集成化环境训练
如果你是在autodl上的机器可以直接使用封装好的镜像创建实例，开箱即用  
如果是本地或者其他服务器上也可以使用，需要手动安装一些pip包

## 注意
本项目仅供用于学习、测试人工智能技术使用  
请勿用于训练生成不良或侵权图片内容

## 关于项目
在autodl封装的镜像名称为：dreambooth-for-diffusion  
可在创建实例时直接选择公开的算法镜像使用。  
在autodl内蒙A区A5000的机器上封装，如遇到问题且无法自行解决的朋友请使用同一环境。  
白菜写教程时做了尽可能多的测试，但仍然无法确保每一个环节都完全覆盖    
如有小错误可尝试手动解决，或者访问git项目地址查看最新的README  
项目地址：https://github.com/CrazyBoyM/dreambooth-for-diffusion

## #强烈建议
1.用vscode的ssh功能远程连接到本服务器，训练体验更好，autodl自带的notebook也不错，有文件上传、下载功能。   
（vscode连接autodl教程：https://www.autodl.com/docs/vscode/ ）  
### 2.(重要)把train文件夹整个移动到/root/autodl-tmp/路径下进行训练(数据盘)，避免系统盘空间满
有的机器数据盘也很小，需要自行关注开合适的机器或进行扩容

如果遇到问题可到b站主页找该教程对应训练演示的视频：https://space.bilibili.com/291593914
（因为现在写时视频还没做 

## 服务器的数据迁移
经常关机后再开机发现机器资源被占用了，这时候你只能另外开一台机器了  
但是对于已经关机的机器在菜单上有个功能是“跨实例拷贝数据”，  
可以很方便地同步/root/autodl-tmp文件夹下的内容到其他已开机的机器（所以推荐工作文件都放这）  
（注意，只适用于同一区域的机器之间）
数据迁移教程：https://www.autodl.com/docs/migrate_instance/

### 本文件为notebook在线运行版
具体详细的教程和参数说明请在根目录下教程.md 文件中查看。  
在notebook中执行linux命令，需要前面加个!(感叹号)  
代码块前如果有个[*]，表示正在运行该步骤，并不是卡住了


# 笔者前言

linux压缩一个文件夹为单个文件包的命令：
```
!zip xx.zip -r ./xxx
```
解压一个包到文件夹：
```
!unzip xx.zip -d xxx
```
或许你在上传、下载数据集时会用到。

其他linux基础命令：https://www.autodl.com/docs/linux/

关于文件上传下载的提速可查看官网文档推荐的几种方式：https://www.autodl.com/docs/scp/

### 首先，进入工作文件夹（记得先把dreambooth-for-diffusion文件夹移动到autodl-tmp目录下）

In [ ]:
%cd /root/autodl-tmp/dreambooth-for-diffusion

# 准备数据集
该部分请参考教程.md文件中的详细内容自行上传并处理你的数据集  
dreambooth-for-diffusion/datasets/test中为16张仅供于学习测试的样本数据，便于你了解以下代码的用处  


## 一键裁剪
### 图像批量center crop裁剪(正方形裁剪)
./datasets/test是原始图片数据文件夹，请上传你的图片数据并进行更换  
width和height请设置为8的整倍数，并记得修改训练脚本中的参数  
（在显存低于20G的设备上请修改使用小于768的分辨率数据去训练，比如512）  
如果是对透明底的png图处理成纯色底可以加--png参数，具体可以看对应的代码文件

In [18]:
!python tools/handle_images.py --origin_image_path  ./datasets/test \
                --output_image_path ./datasets/test2 \
                --width=768 --height=768

最大边同比缩放(后续输入任意比例的图片)   

In [9]:
!python tools/scale_images.py --origin_image_path  ./datasets/test \
                --output_image_path  ./datasets/test5 \
                --max_side_length 712

d:\PC\cyvision\dreambooth-for-diffusion\tools\scale_images.py:28: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.LANCZOS instead.
  resized_image = image.resize((new_width, new_height), Image.ANTIALIAS)


## 一键打标签
### 图像批量自动标注
使用deepdanbooru生成tags标注文件。（仅针对纯二次元类图片效果较好，其他风格请手动标注）  
./datasets/test2中是需要打标注的图片数据，请按需更换为自己的路径 

In [2]:
# 该步根据需要标注文件数量不同，需要运行一段时间（测试6000张图片需要10分钟）
!python tools/label_images.py  --path=./datasets/cyAnime --model_path='./deepdanbooru/'

## 转换ckpt检查点文件为diffusers官方权重
输出的文件在dreambooth-for-diffusion/model下  
./ckpt_models/sd_1-5.ckpt需要更换为你自己的权重文件路径  

如需转换写实风格模型：

In [ ]:
# 该步需要运行大约一分钟 
!python tools/ckpt2diffusers.py \
    --checkpoint_path=./ckpt_models/sd_1-5.ckpt \
    --dump_path=./model \
    --original_config_file=./ckpt_models/model.yaml \
    --scheduler_type="ddim"

如需转换二次元风格模型：

In [ ]:
# 该步需要运行大约一分钟 
!python tools/ckpt2diffusers.py \
    --checkpoint_path=./ckpt_models/nd_lastest.ckpt \
    --dump_path=./model \
    --vae_path=./ckpt_models/animevae.pt \
    --original_config_file=./ckpt_models/model.yaml \
    --scheduler_type="ddim"

对于需要转换某个特殊模型(7g)并遇到报错的同学，ckpt_models里的nd_lastest.ckpt就是你需要的文件。  
如果希望手动转换，我在./tools下放了一份ckpt_prune.py可以参考。

# 训练Unet和text encoder
以下训练脚本会自动帮你启动tensorboard日志监控进程，入口可参考: https://www.autodl.com/docs/tensorboard/  
使用tensorboard面板可以帮助分析loss在不同step的总体下降情况  
如果你嫌输出太长，可以在以下命令每一行后加一句 &> log.txt, 会把输出都扔到这个文件中 
```
!sh train_style.sh &> log.txt
```
本代码包环境已在A5000、3090测试通过，如果你在某些机器上运行遇到问题可以尝试卸载编译的xformers
```
!pip uninstall xformers
```

### 如果需要训练特定人、事物： 
（推荐准备3~5张风格统一、特定对象的图片）  
请打开train_object.sh具体修改里面的参数

In [ ]:
# 大约十分钟后才会在tensorboard有日志（因为前十分钟在生成同类别伪图）
!sh train_object.sh 

In [ ]:
# 事先进行最大边同比缩放后，支持任意尺寸输入(batch_size=1)
!sh train_object_all.sh  

### 如果要训练画风： 
（推荐准备3000+张图片，包含尽可能的多样性，数据决定训练出的模型质量）  
请打开train_object具体修改里面的参数   

In [3]:
# 正常训练立刻就可以在tensorboard看到日志
# 如果输入图像是正方形
!sh train_style.sh  

In [ ]:
# 事先进行最大边同比缩放后，支持任意尺寸输入(batch_size=1)
!sh train_style_all.sh  

后台训练法请参考教程.md中的内容

省钱训练法（训练成功后自动关机，适合步数很大且夜晚训练的场景）

In [6]:
!sh back_train.sh

## 拓展：训练Textual inversion


In [ ]:
!sh train_textual_inversion.sh

### 测试训练效果
打开dreambooth-for-diffusion/test_model.py文件修改其中的model_path和prompt，然后执行以下测试  
会生成一张图片 在左侧test-1、2、3.png

In [4]:
# 大约5~10s 
!python test_model.py

### 转换diffusers官方权重为ckpt检查点文件
输出的文件在dreambooth-for-diffusion/ckpt_models/中，名为newModel.ckpt

原始保存：

In [ ]:
!python tools/diffusers2ckpt.py ./new_model ./ckpt_models/newModel.ckpt 

以下代码添加--half 保存float16半精度，权重文件大小会减半（约2g），效果基本一致

In [ ]:
!python tools/diffusers2ckpt.py ./new_model ./ckpt_models/newModel_half.ckpt --half

下载ckpt文件，去玩吧~

有问题可以进XDiffusion QQ Group：455521885  

### 记得定期清理不需要的中间权重和文件，不然容易导致空间满
大部分问题已在教程.md中详细记录，也包含其他非autodl机器手动部署该训练一体化封装代码包的步骤

In [ ]:
# 清理文件的示例
!rm -rf ./model* # 删除当前目录model文件/文件夹
!rm -rf ./new_* # 删除当前目录所有new_开头的模型文件夹
# !rm -rf ./datasets/test2 #删除datasets中的test2数据集 